# Analyzing Historical Stock/Revenue Data and Building a Dashboard

In [2]:
pip install yfinance
pip install plotly

In [12]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Question 1 - Extracting Tesla Stock Data Using yfinance 

In [69]:
Tesla = yf.Ticker("TSLA")

Tesla_df = Tesla.history(period ="max")

Tesla_df.reset_index(inplace=True)

Tesla_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0
1,2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0
2,2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0
3,2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0
4,2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0


### Question 2 - Extracting Tesla Revenue Data Using Webscraping

In [10]:
url = " https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

data  = requests.get(url).text

In [13]:
soup = BeautifulSoup(data, 'html5lib')

In [36]:
tesla_data = pd.DataFrame(columns=["Date", "Revenue"])


tesla_tables = soup.find_all('table')

for index,table in enumerate(tesla_tables):
    if ("Tesla Quarterly Revenue" in str(table)):
        tesla_table_index = index

# First we isolate the body of the table which contains all the information
# Then we loop through each row and find all the column values for each row
for row in soup.find("tbody").find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text.replace("$", "").replace(",", "")
    # Finally we append the data of each row to the table
    tesla_data = tesla_data.append({"Date" : date, "Revenue" : revenue}, ignore_index=True)

In [73]:
tesla_data.tail()

,Date,Revenue
8,2012,413
9,2011,204
10,2010,117
11,2009,112
12,2008,15


### Question 3 - Extracting GameStop Stock Data Using yfinance

In [70]:
gamestop = yf.Ticker("GME")

gamestop_df = gamestop.history(period ="max")

gamestop_df.reset_index(inplace=True)

gamestop_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,6.480512,6.773398,6.413182,6.766665,19054000,0.0,0.0
1,2002-02-14,6.850829,6.864295,6.682504,6.733002,2755400,0.0,0.0
2,2002-02-15,6.733001,6.749833,6.632006,6.699336,2097400,0.0,0.0
3,2002-02-19,6.665672,6.665672,6.312189,6.430017,1852600,0.0,0.0
4,2002-02-20,6.463680,6.648837,6.413182,6.648837,1723200,0.0,0.0


### Question 4 - Extracting GameStop Revenue Data Using Webscraping

In [72]:
url = " https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

data  = requests.get(url).text

soup = BeautifulSoup(data, 'html5lib')

gamestop_data = pd.DataFrame(columns=["Date", "Revenue"])


gamestop_tables = soup.find_all('table')

for index,table in enumerate(gamestop_tables):
    if ("GameStop Quarterly Revenue" in str(table)):
        gamestop_tables_index = index

# First we isolate the body of the table which contains all the information
# Then we loop through each row and find all the column values for each row
for row in soup.find("tbody").find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text.replace("$", "").replace(",", "")
    # Finally we append the data of each row to the table
    gamestop_data = gamestop_data.append({"Date" : date, "Revenue" : revenue}, ignore_index=True)
    
gamestop_data.tail()

,Date,Revenue
12,2009,8806
13,2008,7094
14,2007,5319
15,2006,3092
16,2005,1843


### Question 5 - Tesla Stock and Revenue Dashboard 

In [66]:
#Define Graphing Function
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", 
                                                                           "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Candlestick(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), open = stock_data.Open, high = stock_data.High, low = stock_data.Low, close = stock_data.Close, name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [67]:
#1
p1 = make_graph(Tesla_df, tesla_data, 'Tesla')
p1

### Question 6 - GameStop Stock and Revenue Dashboard

In [68]:
#2
p2 = make_graph(gamestop_df, gamestop_data, 'GameStop')
p2